In [56]:
import pandas as pd
import plotly.express as px
import numpy as np
from geopy.distance import geodesic
import importlib
import utm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import veda.ais_to_parquet
from veda.ais_to_parquet import fn
import veda.trajectory_segmentation
from veda.trajectory_segmentation import calculate_max_radius, segment_by_stationary_periods
import veda.interpolation
from veda.interpolation import regularize_trajectory, regularize_all_trajectories
import veda.coord_to_utm
from veda.coord_to_utm import to_utm

# Download data

I adjusted the `ais-to-parquet.py` script to also extract ship type. We may consider also including:
- ROT
- Heading
- Destination
- ETA

In [3]:
# fn('data/ais_data/aisdk-2025-11-01.csv', 'data/ais_data/aisdk-2025-11-01.parquet')
# fn('data/ais_data/aisdk-2025-11-02.csv', 'data/ais_data/aisdk-2025-11-02.parquet')
# fn('data/ais_data/aisdk-2025-11-03.csv', 'data/ais_data/aisdk-2025-11-03.parquet')
# fn('data/ais_data/aisdk-2025-11-04.csv', 'data/ais_data/aisdk-2025-11-04.parquet')
# fn('data/ais_data/aisdk-2025-11-05.csv', 'data/ais_data/aisdk-2025-11-05.parquet')
# fn('data/ais_data/aisdk-2025-11-06.csv', 'data/ais_data/aisdk-2025-11-06.parquet')
# fn('data/ais_data/aisdk-2025-11-07.csv', 'data/ais_data/aisdk-2025-11-07.parquet')

In [4]:
# Combine all parquet files into a single df
parquet_files = [
    'data/ais_data/aisdk-2025-11-01.parquet',
    'data/ais_data/aisdk-2025-11-02.parquet'#,
    # 'data/ais_data/aisdk-2025-11-03.parquet',
    # 'data/ais_data/aisdk-2025-11-04.parquet',
    # 'data/ais_data/aisdk-2025-11-05.parquet',
    # 'data/ais_data/aisdk-2025-11-06.parquet',
    # 'data/ais_data/aisdk-2025-11-07.parquet'
]
dfs = [pd.read_parquet(file) for file in parquet_files]
df = pd.concat(dfs, ignore_index=True)

In [7]:
# Drop 'Segment' column (since we define our own segments later)
df = df.drop(columns=['Segment'])

# Preprocessing

In [9]:
print("Total vessels: ", df['MMSI'].unique().shape[0])

cdf = df[df['Ship type'] == 'Cargo'].drop(columns=['Ship type'])
print("Cargo vessels: ", cdf['MMSI'].unique().shape[0])

Total vessels:  1608
Cargo vessels:  562


## Missing Values

Here I check to see where we have missing values. Since it seems that there are only a few vessels missing SOG and COG values, we could either calculate them manually based on lat/long and time, or we could just drop the affected vessels.

In [10]:
cdf.isnull().sum()

Timestamp       0
Latitude        0
Longitude       0
SOG           295
COG          4447
MMSI            0
dtype: int64

In [11]:
m_cog = cdf[cdf['COG'].isnull()]['MMSI'].unique()
m_sog = cdf[cdf['SOG'].isnull()]['MMSI'].unique()

m_missing = set(m_cog).union(set(m_sog))
print("Number of ships with missing COG or SOG: ", len(m_missing))

Number of ships with missing COG or SOG:  27


Here we need to decide whether we want to drop the ships with missing COG/SOG values, or whether we want to estimate them using positional and temporal data. 

**For now I will just drop them.**

In [12]:
cdf_clean = cdf[~cdf['MMSI'].isin(m_missing)].copy()

## Trajectory Segmentation Based on Stationary Periods

Split trajectories whenever the ship is stationary for more than 30 minutes. A ship is considered stationary if:
- SOG < 1 knot (already in m/s after conversion), OR
- Position variance < 50m over the stationary period

In [22]:
importlib.reload(veda.trajectory_segmentation)
from veda.trajectory_segmentation import segment_by_stationary_periods

In [23]:
# Segment trajectories
cdf_segmented = segment_by_stationary_periods(
    cdf_clean,
    sog_threshold=0.5,        # 1 knot in m/s
    position_threshold=50,    # 50 meters
    time_threshold=30         # 30 minutes
)

print(f"Total vessels: {cdf_segmented['MMSI'].nunique()}")
print(f"Total trajectories: {cdf_segmented['Trajectory'].nunique()}")
print(f"Average trajectories per vessel: {cdf_segmented['Trajectory'].nunique() / cdf_segmented['MMSI'].nunique():.2f}")

Total vessels: 530
Total trajectories: 582
Average trajectories per vessel: 1.10


In [26]:
ships_with_multiple_trajectories = cdf_segmented.groupby('MMSI')['Trajectory'].nunique()
multi_traj_ships = ships_with_multiple_trajectories[ships_with_multiple_trajectories > 1]

if len(multi_traj_ships) > 0:
    sample_mmsi = multi_traj_ships.index[3]
    sample_ship_df = cdf_segmented[cdf_segmented['MMSI'] == sample_mmsi].sort_values('Timestamp')
    
    print(f"Visualizing MMSI {sample_mmsi} with {sample_ship_df['Trajectory'].nunique()} trajectories")
    
    # Create color map for trajectories
    fig = px.line_map(
        sample_ship_df,
        lat="Latitude",
        lon="Longitude",
        color="Trajectory",
        hover_data=["Timestamp", "SOG"],
        zoom=5,
        title=f"Segmented Trajectories for MMSI {sample_mmsi}"
    )
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()
else:
    print("No ships with multiple trajectories found")

Visualizing MMSI 211833390 with 2 trajectories


## Time Series Regularization

Resample trajectories to regular time intervals using linear interpolation. This ensures consistent sampling frequency for RNN training.

In [ ]:
importlib.reload(veda.interpolation)
from veda.interpolation import regularize_all_trajectories

In [28]:
INTERVAL_MINUTES = 5 # (I picked 5 arbitrarily)

cdf_regular = regularize_all_trajectories(cdf_segmented, interval_minutes=INTERVAL_MINUTES)

Regularizing 582 trajectories...
  Processed 100/582 trajectories
  Processed 200/582 trajectories
  Processed 300/582 trajectories
  Processed 400/582 trajectories
  Processed 500/582 trajectories
  Completed all 582 trajectories


In [30]:
sample_traj = cdf_regular[cdf_regular['Trajectory'] == 50].sort_values('Timestamp')
time_diffs = sample_traj['Timestamp'].diff().dt.total_seconds() / 60
print(f"\nMean interval: {time_diffs.mean():.2f} minutes")


Mean interval: 5.00 minutes


## Convert to UTM Coordinates

Convert latitude/longitude to UTM (as they did in the paper)

In [32]:
importlib.reload(veda.coord_to_utm)
from veda.coord_to_utm import to_utm

In [33]:
cdf_utm = to_utm(cdf_regular)

# Feature Selection

Features we'll definitely use:
- UTM coordinates
- SOG
- COG (decomposed into east/north velocities)

Others we could consider:
- Spatial context
    - Acceleration
    - Heading
    - ROT
    - Distance from coast/port/shipping lanes
    - Water depth
- Temporal context
    - Tidal data
    - Time of day
    - Day of week
    - Season

In [36]:
# Decompose COG into its vector components
cog_radians = np.radians(cdf_utm['COG'])                 # convert to radians
cdf_utm['v_east'] = cdf_utm['SOG'] * np.sin(cog_radians)      # eastward component
cdf_utm['v_north'] = cdf_utm['SOG'] * np.cos(cog_radians)     # northward component

In [41]:
cdf_utm.columns

Index(['Timestamp', 'Latitude', 'Longitude', 'SOG', 'COG', 'MMSI',
       'Trajectory', 'UTM_x', 'UTM_y', 'UTM_zone', 'UTM_letter', 'v_east',
       'v_north'],
      dtype='object')

In [42]:
# Split data into different trajectories, keeping only relevant features
fdf = cdf_utm[['Trajectory', 'Timestamp', 'UTM_x', 'UTM_y', 'SOG', 'v_east', 'v_north']].copy()

trajectories = []
    
for traj_id in fdf['Trajectory'].unique():
    traj_data = fdf[fdf['Trajectory'] == traj_id].sort_values('Timestamp')
    features = traj_data[['UTM_x', 'UTM_y', 'SOG', 'v_east', 'v_north']].values
    trajectories.append(features)

## Train-Test-Val Split

I just did a simple random sample with 70% train, 15% test, 15% val

In [50]:
train, temp = train_test_split(trajectories, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

## Normalisation

Options to consider:
- min-max scaling
- z-score
- feature-specific scaling
    - I think min-max is appropriate for the simple case in which we just look at UTM, SOG, and COG, but that might change if we add more features

In [ ]:
# Fit min-max scaler to training set
train_stacked = np.vstack(train)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train_stacked)

,feature_range,"(-1, ...)"
,copy,True
,clip,False


In [55]:
# Apply min-max scaling to each set
train_s = [scaler.transform(traj) for traj in train]
val_s = [scaler.transform(traj) for traj in val]
test_s = [scaler.transform(traj) for traj in test]

### Set up dataset/dataloader

In [57]:
# Define Trajectory Dataset
class TrajectoryDataset(Dataset):
    def __init__(self, trajectories):
        self.trajectories = trajectories
    
    def __len__(self):
        return len(self.trajectories)
    
    def __getitem__(self, idx):
        traj = torch.FloatTensor(self.trajectories[idx])
        return traj

In [58]:
def pad_trajectories(batch):
    lengths = torch.tensor([len(traj) for traj in batch])
    padded = pad_sequence(batch, batch_first=True, padding_value=0.0)

    lengths, perm_idx = lengths.sort(descending=True)
    padded = padded[perm_idx]

    return padded, lengths

In [59]:
train_dataset = TrajectoryDataset(train_s)
val_dataset = TrajectoryDataset(val_s)
test_dataset = TrajectoryDataset(test_s)

batch_size = 32 # we need to pick one, this is arbitrary

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_trajectories)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_trajectories)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_trajectories)

In [62]:
# Check to see what's goin' on
for batch, lengths in train_loader:
    print(f"Batch shape: {batch.shape}")  # (batch_size, max_length, 5)
    print(f"Lengths: {lengths}")  # (batch_size,)
    break

Batch shape: torch.Size([32, 576, 5])
Lengths: tensor([576, 551, 533, 508, 416, 404, 400, 388, 290, 256, 248, 242, 236, 230,
        226, 222, 221, 203, 170, 141, 125, 123, 121, 104,  98,  93,  84,  76,
         65,  52,  48,  41])
